# Customer Analytics : Prep Data For Modelling

## Background :
A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called Training Data Ltd. to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to customer_train.csv, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

## Instructions :
The Head Data Scientist at Training Data Ltd. has asked you to create a DataFrame called ds_jobs_transformed that stores the data in customer_train.csv much more efficiently. Specifically, they have set the following requirements:

* Columns containing categories with only two factors must be stored as Booleans (bool).
* Columns containing integers only must be stored as 32-bit integers (int32).
* Columns containing floats must be stored as 16-bit floats (float16).
* Columns containing nominal categorical data must be stored as the category data type.
* Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

* The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

## Import necessary libraries

In [1]:
import pandas as pd
import os

pd.set_option('display.max_colwidth', None)

pwd = os.getcwd()
filepath = os.path.join(pwd, 'customer_train.csv')

# Load the dataset
ds_jobs = pd.read_csv(filepath)

# View first 3 rows of the dataset
ds_jobs.head(3)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


## Quick EDA to understand the dataset

### Numerical Columns
First, let's identify the numerical columns by examing dtype

In [2]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

ds_jobs_transformed.dtypes.sort_values()

student_id                  int64
training_hours              int64
city_development_index    float64
job_change                float64
city                       object
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
dtype: object

* `student_id` & `training_hours` are integers and not in **int64** format
* `city_development_index` & `job_change` are integers and not in **float16** format

### Categorical columns
Next, let's take a closer look at the categorical columns to understand if they should be :
* Nominal
* Ordinal
* Bool

In [3]:
# EDA to help identify ordinal, nominal, and two-factor categories
cat_feat = pd.DataFrame(columns=['Col_name', 'nunique', 'values' ])

for index, value in enumerate(ds_jobs_transformed.select_dtypes('object')) :
    cat_feat.loc[index] = [value, ds_jobs_transformed[value].nunique(), ds_jobs_transformed[value].unique()]

cat_feat

,Col_name,nunique,values
0,city,123,"[city_103, city_40, city_21, city_115, city_162, city_176, city_160, city_46, city_61, city_114, city_13, city_159, city_102, city_67, city_100, city_16, city_71, city_104, city_64, city_101, city_83, city_105, city_73, city_75, city_41, city_11, city_93, city_90, city_36, city_20, city_57, city_152, city_19, city_65, city_74, city_173, city_136, city_98, city_97, city_50, city_138, city_82, city_157, city_89, city_150, city_70, city_175, city_94, city_28, city_59, city_165, city_145, city_142, city_26, city_12, city_37, city_43, city_116, city_23, city_99, city_149, city_10, city_45, city_80, city_128, city_158, city_123, city_7, city_72, city_106, city_143, city_78, city_109, city_24, city_134, city_48, city_144, city_91, city_146, city_133, city_126, city_118, city_9, city_167, city_27, city_84, city_54, city_39, city_79, city_76, city_77, city_81, city_131, city_44, city_117, city_155, city_33, city_141, city_127, city_62, ...]"
1,gender,3,"[Male, nan, Female, Other]"
2,relevant_experience,2,"[Has relevant experience, No relevant experience]"
3,enrolled_university,3,"[no_enrollment, Full time course, nan, Part time course]"
4,education_level,5,"[Graduate, Masters, High School, nan, Phd, Primary School]"
5,major_discipline,6,"[STEM, Business Degree, nan, Arts, Humanities, No Major, Other]"
6,experience,22,"[>20, 15, 5, <1, 11, 13, 7, 17, 2, 16, 1, 4, 10, 14, 18, 19, 12, 3, 6, 9, 8, 20, nan]"
7,company_size,8,"[nan, 50-99, <10, 10000+, 5000-9999, 1000-4999, 10-49, 100-499, 500-999]"
8,company_type,6,"[nan, Pvt Ltd, Funded Startup, Early Stage Startup, Other, Public Sector, NGO]"
9,last_new_job,6,"[1, >4, never, 4, 3, 2, nan]"


**Ordinal** <br>
since they've have a natural order
- `enrolled_university`, `education_level`, `experience`, `company_size`, `last_new_job`

**Nominal** <br>
no natural order and more than 2 unique factors
- `gender`, `city`, `major_discipline`, `company_type`

**bool** <br>
only 2 unique factors
- `relevant_experience` `job_change`

## Updating dtypes to optimize data size and filtering per requests 
- '>=10 years of experience'
- '>=1000 employees'

In [4]:
ordinal_cat = {
    'enrolled_university' : ["no_enrollment", "Part time course", "Full time course"],
    'education_level' : ["Primary School", "High School", "Graduate","Masters","Phd"],
    'experience' : ['<1'] + list(map(str, range(1,21))) +['>20'],
    'company_size' : ["<10", "10-49", "50-99", "100-499", "500-999", "1000-4999", "5000-9999", "10000+"],
    'last_new_job' : ["1", "2", "3", "4", ">4","never"]
}

nominal_cat = ['gender', 'city', 'major_discipline', 'company_type']
boolean = ['relevant_experience', 'job_change']
integer = ['student_id', 'training_hours']
float_values = ['city_development_index']


for col in ds_jobs_transformed.columns :
    # ordinal 
    if col in ordinal_cat.keys() :
        ds_jobs_transformed[col] = pd.Categorical(ds_jobs_transformed[col], categories=ordinal_cat[col], ordered=True)

    # nominal
    elif col in nominal_cat :
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
        
    # bool
    elif col in boolean :
        ds_jobs_transformed[col] = ds_jobs_transformed[col].apply(lambda x : 1 if x in (['Has relevant experience','1',1]) else 0).astype(bool)

    # int
    elif col in integer :
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
        
    # float
    elif col in float_values :
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')


ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]

ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
31,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
34,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
40,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


## Check memory usage **before** & **after** transformation

In [5]:
before_total = ds_jobs.memory_usage(deep=True).sum()/(1024 * 1024)
before_by_col = round(ds_jobs.memory_usage(deep=True) / (1024 * 1024),3)

print(f'Total memory (MB) used before transformation - {before_total:.3f}')
print('\n')
print('Memory by columns')
print(before_by_col)

Total memory (MB) used before transformation - 11.818


Memory by columns
Index                     0.000
student_id                0.146
city                      1.179
city_development_index    0.146
gender                    0.992
relevant_experience       1.457
enrolled_university       1.279
education_level           1.175
major_discipline          1.045
experience                1.070
company_size              0.976
company_type              0.999
last_new_job              1.062
training_hours            0.146
job_change                0.146
dtype: float64


In [6]:
after_total = ds_jobs_transformed.memory_usage(deep=True).sum()/ (1024 * 1024)
total_save = before_total-after_total
total_save_pct = (before_total-after_total)/before_total

print(f'Total memory (MB) used after transformation - {after_total:.3f}')
print('\n')
print('Memory by columns')
print(round(ds_jobs_transformed.memory_usage(deep=True)/ (1024 * 1024),3))
print('\n')
print(f'total saved : {total_save:.2f}(MB) (-{total_save_pct:.2f}%)')

Total memory (MB) used after transformation - 0.078


Memory by columns
Index                     0.017
student_id                0.008
city                      0.014
city_development_index    0.004
gender                    0.002
relevant_experience       0.002
enrolled_university       0.002
education_level           0.003
major_discipline          0.003
experience                0.004
company_size              0.003
company_type              0.003
last_new_job              0.003
training_hours            0.008
job_change                0.002
dtype: float64


total saved : 11.74(MB) (-0.99%)
